# GoDaddy - Microbusiness Density Forecasting

In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
from sklearn.ensemble import VotingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer    
from sklearn.ensemble import VotingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer   
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

BASE = 'data/'
os.environ["CUDA_VISIBLE_DEVICES"]="0"

ACT_THR = 150       # lastactive threshold for training
MONTH_1 = 39        # Nov and Dec of 2022
MONTH_last = 40     # Nov and Dec of 2022

IS_DEBUG = True     # use toy data for debugging


# Load Data

In [2]:
# competition dataset
census = pd.read_csv(BASE + 'godaddy-microbusiness-density-forecasting/census_starter.csv')
train = pd.read_csv(BASE + 'godaddy-microbusiness-density-forecasting/train.csv')
reaveal_test = pd.read_csv(BASE + 'godaddy-microbusiness-density-forecasting/revealed_test.csv')
train = pd.concat([train, reaveal_test]).sort_values(by=['cfips','first_day_of_month']).reset_index()
test = pd.read_csv(BASE + 'godaddy-microbusiness-density-forecasting/test.csv')
sub = pd.read_csv(BASE + 'godaddy-microbusiness-density-forecasting/sample_submission.csv')

# external dataset
# lng and lat
coords = pd.read_csv(BASE + "cfips_location.csv")
# CPI
cpi= pd.read_csv(BASE + 'go-daddy-cpi/go_daddy_cpi - Sheet1.csv')

# CO-EST2021-ALLDATA: 
# Annual Resident Population Estimates, Estimated Components of Resident Population Change, 
# and Rates of the Components of Resident Population Change for States and Counties:
# https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2021/CO-EST2021-ALLDATA.pdf
co_est = pd.read_csv(BASE + "/us-indicator/co-est2021-alldata.csv", encoding='latin-1')
co_est["cfips"] = co_est.STATE*1000 + co_est.COUNTY
co_columns = ['SUMLEV','DIVISION','ESTIMATESBASE2020','POPESTIMATE2020','POPESTIMATE2021',
              'NPOPCHG2020','NPOPCHG2021','BIRTHS2020','BIRTHS2021','DEATHS2020','DEATHS2021',
              'NATURALCHG2020','NATURALCHG2021','INTERNATIONALMIG2020','INTERNATIONALMIG2021',
              'DOMESTICMIG2020','DOMESTICMIG2021','NETMIG2020','NETMIG2021','RESIDUAL2020',
              'RESIDUAL2021','GQESTIMATESBASE2020','GQESTIMATES2020','GQESTIMATES2021',
              'RBIRTH2021','RDEATH2021','RNATURALCHG2021','RINTERNATIONALMIG2021','RDOMESTICMIG2021','RNETMIG2021']

# Preprocess

In [3]:
# label holdout set (not initially available)
drop_index = (test.first_day_of_month == '2022-11-01') | (test.first_day_of_month == '2022-12-01')
test = test.loc[~drop_index,:]

train['istest'] = 0
test['istest'] = 1

# concat data for better FE
raw = pd.concat((train, test)).sort_values(['cfips','row_id']).reset_index(drop=True)
raw = raw.merge(coords.drop("name", axis=1), on="cfips")
raw = raw.merge(census, on="cfips", how="left")
raw = raw.merge(co_est, on="cfips", how="left")

if IS_DEBUG: # use only partial data for faster debugging 
    raw = raw.query('cfips == 1001')

raw.shape

(47, 71)

## remove anomaly


In [4]:
for cfip in tqdm(raw.cfips.unique()): 
    indices = (raw['cfips'] == cfip) 
    
    tmp = raw.loc[indices].copy().reset_index(drop=True)
    val = tmp.microbusiness_density.values.copy()
    
    for i in range(37, 2, -1): # only in training set 
        thr = 0.10 * np.mean(val[:i]) # use 10% of rolling avg as threshold
        
        val_diff = val[i] - val[i - 1] 
        if (val_diff >= thr) or (val_diff <= -thr):              
            if val_diff > 0:
                val[:i] += val_diff - 0.003 
            else:
                val[:i] += val_diff + 0.003  

    val[0] = val[1] * 0.99 # smooth
    raw.loc[indices, 'microbusiness_density'] = val

  0%|          | 0/1 [00:00<?, ?it/s]

## SMAPE is a relative metric so target must be converted.

In [5]:
raw['target'] = raw.groupby('cfips')['microbusiness_density'].shift(-1)
raw['target'] = raw['target']/raw['microbusiness_density'] - 1
raw['lastactive'] = raw.groupby('cfips')['active'].transform('last')

# outlier cleaning
raw.loc[raw['cfips']==28055, 'target'] = 0.0
raw.loc[raw['cfips']==48269, 'target'] = 0.0

# Feature Engineering

## Basic FE

In [6]:
raw['first_day_of_month'] = pd.to_datetime(raw["first_day_of_month"])

raw['state_i1'] = raw['state'].astype('category')
raw['county_i1'] = raw['county'].astype('category')

raw['state_i'] = raw['state'].factorize()[0]
raw['county_i'] = (raw['county'] + raw['state']).factorize()[0]

raw['state'] = raw.groupby('cfips')['state'].ffill()
raw['county'] = raw.groupby('cfips')['county'].ffill()

raw['scale'] = (raw['first_day_of_month'] - raw['first_day_of_month'].min()).dt.days
raw['scale'] = raw['scale'].factorize()[0]
raw["dcount"] = raw.groupby(['cfips'])['row_id'].cumcount()

## Lag feature

In [7]:
def build_features(raw, target='microbusiness_density', target_act='active_tmp', lags = 6):
    feats = []   

    for lag in range(1, lags):
        raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')[target].shift(lag)
        feats.append(f'mbd_lag_{lag}')
        
    lag = 1
    for window in [2, 4, 6, 8, 10]:
        raw[f'mbd_rollmea{window}_{lag}'] = raw.groupby('cfips')[f'mbd_lag_{lag}'].transform(lambda s: s.rolling(window, min_periods=1).sum())        
        feats.append(f'mbd_rollmea{window}_{lag}')
    
    census_columns = list(census.columns)
    census_columns.remove( "cfips")
    
    feats += census_columns
    feats += co_columns
    return raw, feats

In [8]:
raw, feats = build_features(raw, 'target', 'active', lags = 9)
features = ['state_i']
features += feats
features += ['lng','lat','scale']

## Latitude and Longitude
Latitude and Longitude feature engineering from [Positional Encoder Graph Neural Networks for Geographic Data](!https://arxiv.org/abs/2111.10144) and [here](!https://www.kaggle.com/competitions/playground-series-s3e1/discussion/376210).

In [9]:
coordinates = raw[['lng', 'lat']].values

# Encoding tricks
emb_size = 20
precision = 1e6

latlon = np.expand_dims(coordinates, axis=-1)

m = np.exp(np.log(precision) / emb_size)
angle_freq = m ** np.arange(emb_size)
angle_freq = angle_freq.reshape(1,1, emb_size)
latlon = latlon * angle_freq
latlon[..., 0::2] = np.cos(latlon[..., 0::2])

In [10]:
def rot(df):
    for angle in [15, 30, 45]:
        df[f'rot_{angle}_x'] = (np.cos(np.radians(angle)) * df['lat']) + \
                                (np.sin(np.radians(angle)) * df['lng'])
        
        df[f'rot_{angle}_y'] = (np.cos(np.radians(angle)) * df['lat']) - \
                                (np.sin(np.radians(angle)) * df['lng'])
        
    return df

raw = rot(raw)

In [11]:
features += ['rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'rot_45_y']

## CPI

In [12]:
raw['year']=raw.first_day_of_month.dt.year
raw['month']=raw.first_day_of_month.dt.month

cpi['category'] = pd.to_datetime(cpi['category'] )
cpi["year"] = cpi['category'] .dt.year
cpi["month"] = cpi['category'] .dt.month
cpi["dcount"] = cpi.index

# Linear prediction for future CPI from 2022-05-01
result_list = []
pred_cpi_set = cpi[cpi.category>='2022-05-01']

for col in pred_cpi_set.columns[1:-3]: # exclude first_day_of_month, year, month, dcount
    X = np.array(range(1,10)).reshape(-1, 1)
    X_pred = np.array(range(10,15)).reshape(-1, 1)

    Y = pred_cpi_set[col].values[:9]
    
    # linear prediction
    linear_regressor = LinearRegression()  
    linear_regressor.fit(X, Y)  
    Y_pred = linear_regressor.predict(X_pred)

    result_list.append(Y_pred)

# fill future val with prediction
for i,col in enumerate(cpi.columns[1:-3]):
    cpi.loc[42:,col] = result_list[i]

cpi.rename(columns = {'category':'first_day_of_month'}, inplace = True)

# merge into the originalset 
raw = raw.merge(cpi, on=['first_day_of_month','year','month','dcount'], how='left')

features += ['electricity_cpi','fuel_cpi','gas_all_cpi','gas_unleaded_cpi','cpi_all_urban']

# Modeling

In [13]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

def vsmape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * smap

In [14]:
def get_model():
    cat_model = cat.CatBoostRegressor(
        iterations=2000,
        loss_function="MAPE",
        verbose=0,
        grow_policy='SymmetricTree',
        learning_rate=0.035,
        colsample_bylevel=0.8,
        max_depth=5,
        l2_leaf_reg=0.2,
        subsample=0.70,
        max_bin=4096,
    )

    return cat_model


def base_models():
    lgb_model = lgb.LGBMRegressor(n_iter=300,
                                boosting_type='dart',
                                verbosity=-1,
                                objective='l1',
                                random_state=42,
                                colsample_bytree=0.88,
                                colsample_bynode=0.1,
                                max_depth=8,
                                learning_rate=0.0036,
                                lambda_l2=0.5,
                                num_leaves=61,
                                seed=42,
                                min_data_in_leaf=213)
    
    xgb_model = xgb.XGBRegressor(
                                objective='reg:squarederror',
                                tree_method="hist",
                                n_estimators=795,
                                learning_rate=0.0075,
                                max_depth=6,  
                                subsample=0.50,
                                colsample_bytree=0.50,
                                max_bin=4096,
                                n_jobs=-1)


    cat_model = cat.CatBoostRegressor(
                                    iterations=2000,
                                    loss_function="MAPE",
                                    verbose=0,
                                    grow_policy='SymmetricTree',
                                    learning_rate=0.035,
                                    colsample_bylevel=0.8,
                                    max_depth=5,
                                    l2_leaf_reg=0.2,
                                    subsample=0.70,
                                    max_bin=4096)
    

    
    rf_model=RandomForestRegressor(criterion="friedman_mse",
                                    n_estimators= 500,
                                    max_depth=5,
                                    max_leaf_nodes=17,
                                    n_jobs=-1,
                                    random_state=42)
    models = {}
    models['xgb'] = xgb_model
    models['lgbm'] = lgb_model
    models['cat'] = cat_model
    models['rf']=rf_model

    return models

In [15]:
# init
raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
raw['k'] = 1.
raw['microbusiness_density'].fillna(0, inplace = True)

for TS in range(MONTH_1, MONTH_last):
    print(f'TS: {TS}')
   
   # init model
    models = base_models()
    model0 = models['xgb']
    # model1 = models['lgbm']
    model2 = models['cat']
    model3 = models['rf']

    # train val split
    train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 1) & (raw.lastactive>ACT_THR) 
    valid_indices = (raw.istest==0) & (raw.dcount == TS) 
    
    # fit model
    # Train each of the models on the current TS
    model0.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.002, 0.006))
    
    # model1.fit(
    #     raw.loc[train_indices, features],
    #     raw.loc[train_indices, 'target'].clip(-0.002, 0.006))
    
    model2.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.002, 0.006))
    
    model3.fit(
        raw.loc[train_indices, features].fillna(method='bfill'),
        raw.loc[train_indices, 'target'].clip(-0.002, 0.006))    
    

    # stacking 
    # build input for stacking model prediction
    tr_pred0 = model0.predict(raw.loc[train_indices, features])
    # tr_pred1 = model1.predict(raw.loc[train_indices, features])
    tr_pred2 = model2.predict(raw.loc[train_indices, features])
    tr_pred3 = model3.predict(raw.loc[train_indices, features].fillna(method='bfill'))
    
    train_preds = np.column_stack((tr_pred0, tr_pred2,tr_pred3))
    # train_preds = np.column_stack((tr_pred0, tr_pred1, tr_pred2,tr_pred3))


    # build input for stacking model prediction
    val_preds0 = model0.predict(raw.loc[valid_indices, features])
    # val_preds1 = model1.predict(raw.loc[valid_indices, features])
    val_preds2 = model2.predict(raw.loc[valid_indices, features])
    val_preds3 = model3.predict(raw.loc[valid_indices, features].fillna(method='bfill'))
    
    valid_preds = np.column_stack((val_preds0, val_preds2, val_preds3))
    # valid_preds = np.column_stack((val_preds0, val_preds1, val_preds2,val_preds3))


    # init and fit 
    meta_model = get_model() 
    meta_model.fit(train_preds, raw.loc[train_indices, 'target'].clip(-0.002, 0.006))
    

    # get stacking model prediction 
    ypred = meta_model.predict(valid_preds)
    
    # transform % change back to actual density 
    raw.loc[valid_indices, 'k'] = ypred + 1
    raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

    # Validate
    lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
    dt = raw.loc[raw.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
    df = raw.loc[raw.dcount==(TS+1), 
                 ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
    
    # map val 
    df['pred'] = df['cfips'].map(dt)
    df['lastval'] = df['cfips'].map(lastval)
    
    # Setting predictions to last values for inactive data points
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
        
    raw.loc[raw.dcount==(TS+1), 'ypred'] = df['pred'].values
    raw.loc[raw.dcount==(TS+1), 'ypred_last'] = df['lastval'].values
    
    # print eval res 
    print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
    print('SMAPE:', smape(df['microbusiness_density'], df['pred']))
    print()


# eval range 
ind = (raw.dcount > MONTH_1)&(raw.dcount <= MONTH_last)

print('Last Value SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred_last']))
print('SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred']))

TS: 39


Last Value SMAPE: 0.8168778492244758
SMAPE: 0.5017516202987414

Last Value SMAPE: 0.8168778492244758
SMAPE: 0.5017516202987414


## keep working on the rest 

In [16]:
TS = 40
print(TS)

train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 1) & (raw.lastactive>ACT_THR) 
valid_indices = (raw.dcount == TS)

model0.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.002, 0.006))
    
# model1.fit(
#     raw.loc[train_indices, features],
#     raw.loc[train_indices, 'target'].clip(-0.002, 0.006))

model2.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, 'target'].clip(-0.002, 0.006))

model3.fit(
    raw.loc[train_indices, features].fillna(method='bfill'),
    raw.loc[train_indices, 'target'].clip(-0.002, 0.006))

# Stacking
tr_pred0 = model0.predict(raw.loc[train_indices, features])
# tr_pred1 = model1.predict(raw.loc[train_indices, features])
tr_pred2 = model2.predict(raw.loc[train_indices, features])
tr_pred3 = model3.predict(raw.loc[train_indices, features].fillna(method='bfill'))
train_preds = np.column_stack((tr_pred0, tr_pred2,tr_pred3))
# train_preds = np.column_stack((tr_pred0, tr_pred1, tr_pred2,tr_pred3))

meta_model = get_model() 
meta_model.fit(train_preds, raw.loc[train_indices, 'target'].clip(-0.002, 0.006))


40


# Recusive Prediction

In [17]:
def build_pred_features(raw, target='microbusiness_density', target_act='active_tmp', lags = 6):

    for lag in range(1, lags):
        raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')[target].shift(lag)
#         raw[f'act_lag_{lag}'] = raw.groupby('cfips')[target_act].diff(lag)
        
    lag = 1
    for window in [2, 4, 6, 8, 10]:
        raw[f'mbd_rollmea{window}_{lag}'] = raw.groupby('cfips')[f'mbd_lag_{lag}'].transform(lambda s: s.rolling(window, min_periods=1).sum())        
    return raw

In [18]:
def predict(raw,model,features):
    pred_months=[
                '2022-12-01',
                '2023-01-01', 
                '2023-02-01', 
                '2023-03-01', 
                '2023-04-01', 
                '2023-05-01', 
                '2023-06-01'
    ]
    
    df_preds=raw.sort_values('first_day_of_month').reset_index(drop=True)
    df_new=raw.copy()

    for i,cur_month in enumerate(pred_months):
        if i: # starting from second one, we need to calculate mbd first 
            df_pred=df_new.query('first_day_of_month==@cur_month')  # query next month for prediction
            df_t=df_new.query('first_day_of_month==@prev_month')
            df_t['next_mbd']=df_t['microbusiness_density']*(1+df_t['target'])

            d=df_t[['cfips','next_mbd']].set_index('cfips').to_dict()
            df_pred['microbusiness_density']=df_pred['cfips'].map(d['next_mbd'])

            indices=df_pred.index
            df_new.loc[indices,'microbusiness_density']=df_pred['microbusiness_density']

        df_new = build_pred_features(df_new, 'target', 'active', lags = 9)
        df_pred=df_new.query('first_day_of_month==@cur_month')  # query next month for prediction

        # prediction
        val_preds0 = model0.predict(df_pred[features])
        # val_preds1 = model1.predict(df_pred[features])
        val_preds2 = model2.predict(df_pred[features])
        val_preds3 = model3.predict(df_pred[features].fillna(method='bfill'))
        valid_preds = np.column_stack((val_preds0, val_preds2, val_preds3))
        # valid_preds = np.column_stack((val_preds0, val_preds1, val_preds2,val_preds3))

        pred = meta_model.predict(valid_preds)

        
        indices=df_pred.index
        df_new.loc[indices,'target']=pred

        prev_month=cur_month
    
    return df_new

raw_pred=predict(raw,model0,features)

# Postprocessing (Recursive)

In [19]:
lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
lastactive = raw.loc[raw.dcount==TS, ['cfips', 'active']].set_index('cfips').to_dict()['active']

df = raw_pred.loc[(raw_pred.dcount>=(TS+1)), ['cfips', 'microbusiness_density']]#.to_dict()['microbusiness_density']
# df = raw_pred.loc[raw_pred.dcount>=TS, ['cfips', 'microbusiness_density']]#.to_dict()['microbusiness_density']
df['lastval'] = df['cfips'].map(lastval)
df['lastactive'] = df['cfips'].map(lastactive)

df.loc[df['lastactive']<=ACT_THR, 'microbusiness_density'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']

raw.loc[raw.dcount>=(TS+1), 'ypred'] = df['microbusiness_density'].values
raw.loc[raw.dcount>=(TS+1), 'ypred_last'] = df['lastval'].values

raw.loc[raw['cfips']==28055, 'microbusiness_density'] = 0
raw.loc[raw['cfips']==48269, 'microbusiness_density'] = 1.762115

# Population Adjustment

In [20]:
COLS = ['GEO_ID','NAME','S0101_C01_026E']
df2020 = pd.read_csv(BASE + 'census-data-for-godaddy/ACSST5Y2020.S0101-Data.csv', usecols=COLS, dtype = 'object')
df2021 = pd.read_csv(BASE + 'census-data-for-godaddy/ACSST5Y2021.S0101-Data.csv',usecols=COLS, dtype = 'object')

df2020 = df2020.iloc[1:]
df2020 = df2020.astype({'S0101_C01_026E':'int'})

df2021 = df2021.iloc[1:]
df2021 = df2021.astype({'S0101_C01_026E':'int'})

df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
df2020.set_index('cfips',inplace=True)
# adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
df2021.set_index('cfips',inplace=True)
# adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

# df2020['mnoshitel'] = df2020['S0101_C01_026E'] / df2021['S0101_C01_026E']
# df2020 = df2020[['cfips','mnoshitel']]



df2020['ppl_2020']=df2020['S0101_C01_026E']
df2020['ppl_2021']=df2021['S0101_C01_026E'].sample(len(df2020))

df2020 = df2020[['ppl_2020','ppl_2021']]

In [21]:
raw = raw.join(df2020, on='cfips')
maska = (raw["first_day_of_month"]>='2023-01-01')
raw.loc[maska, 'microbusiness_density'] = np.round(raw.loc[maska, 'ypred'] * raw.loc[maska, 'ppl_2020'])/raw.loc[maska, 'ppl_2021']
# raw.drop(columns = 'mnoshitel' , inplace = True)
raw.drop(columns = ['ppl_2020','ppl_2021'] , inplace = True)

# Build Submission

In [22]:
test = raw[raw.first_day_of_month >= '2022-11-01'].copy()
test = test[['row_id', 'cfips', 'microbusiness_density']]
test = test[['row_id', 'microbusiness_density']]
test.to_csv('submission.csv', index=False)
test.head(40)

,row_id,microbusiness_density
39,1001_2022-11-01,3.442677
40,1001_2022-12-01,3.470915
41,1001_2023-01-01,3.331248
42,1001_2023-02-01,3.337819
43,1001_2023-03-01,3.354404
44,1001_2023-04-01,3.369076
45,1001_2023-05-01,3.380215
46,1001_2023-06-01,3.399140
